## Acceso a los datos de la web

### Web Scraping

El __Web Scraping__ (o Scraping) son un conjunto de técnicas que se utilizan para obtener de forma automática el contenido que hay en páginas web a través de su código HTML. 
* Es una opción cuando no hay API's para extraer datos de la web
* Otros términos:
    * "Spydering the web"
    * "Web crawling"

Las técnicas de Scraping se pueden enmarcar dentro del campo del Big Data en la primera fase de recolección de datos para su posterior almacenamiento, tratamiento y visualización.

El uso de estas técnicas tienen como finalidad recopilar grandes cantidades de datos de diferentes páginas web cuyo uso posterior puede ser muy variado:

  * homogenización de datos, 
  * tratamiento de contenido para la extracción de conocimiento, 
  * complementar datos en una web, etc.




__Realizar una petición__

La página web con la que vamos a jugar es: [Estadística con datos de movilens](http://gpd.sip.ucm.es/yolanda/formacionContinua/EV2016_pelis.html)
Para acceder al contenido de una página web usamos el protocolo HTTP Request/Response. Python es capaz de hacer una conexión HTTP para acceder a una página web y extraer información.


In [1]:
import requests

url = "http://gpd.sip.ucm.es/yolanda/formacionContinua/EV2016_pelis.html"

# Realizamos la petición HTTP a la web
response = requests.get(url)


* La función __get__ del módulo __request__ de Python abre una conexión con el servidor donde se encuentra la url (parámetro de  __get__) y manda la petición.

* La respuesta del servidor se almacena en la variable (u objeto) _response_.

* A partir del objeto _response_, que almacena muchos datos relacionados con la petición HTTP (cabeceras, cookies, etc.) obtenemos el __status_code__ y el __HTML__ (como un string) de la web. 

In [2]:
response.status_code        # atributo status_code: nos indicará si la petición 
                            # ha tenido éxito

200

In [3]:
# Comprobamos que la petición nos devuelve un Status Code = 200
statusCode = response.status_code
if statusCode == 200:
    print('La petición ha ido bien')
else:
    print('Problemas con la petición...')


La petición ha ido bien


In [4]:
htmlText = response.text    # El atributo text: de tipo string. 
                            # Cadena que contiene todo el html

__Extraer información__

A partir del string __htmlText__ que representa el código interno de la pagina web, parseamos la web para extraer la información que nos interesa.

Mecanismos:

* Usar expresiones regulares para localizar cadenas: esto puede ser muy laborioso, y pesado ...
    
    
    
* Usar alguna de las librerías de Python que permiten parsear el texto HTML de forma cómoda. 
    * __BeautifullSoup__ 
    * __Scrapy__
    
    
__BeautifulSoup__  nos aporta los métodos necesarios (y muy bien optimizados) para obtener el contenido que hay entre las etiquetas HTML.

In [5]:
from bs4 import BeautifulSoup

# Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
soup = BeautifulSoup(htmlText, 'html.parser')

#### 1. Recuperar los enlaces que aparezcan en la página

Los enlaces en HTML se crean mediante la etiqueta `<a>` (su nombre viene del inglés "anchor", literalmente traducido como "ancla").
* El atributo más importante de la etiqueta `<a>` es _href_, que se utiliza para indicar la URL a la que apunta el enlace. 
* Las URL de los enlaces pueden ser internas y externas.

El siguiente ejemplo muestra un enlace a una URL externa:

```<a href="http://grouplens.org/datasets/movielens/">MovieLens Dataset</a>```

__Ejemplo:__ Buscar todos los enlaces que aparecen en una página:

In [11]:

enlaces = soup.find_all("a")

print ('----')
for links in enlaces:
    print ('links: ', links) 
    print ('url: ', links.get('href'))
    print ('content: ', links.contents[0])
    print ('attributes: ', links.attrs)
    print ('----')

----
links:  <a class="anchor-link" href="#Movilens-1M">¶</a>
url:  #Movilens-1M
content:  ¶
attributes:  {'class': ['anchor-link'], 'href': '#Movilens-1M'}
----
links:  <a href="http://grouplens.org/datasets/movielens/">http://grouplens.org/datasets/movielens/</a>
url:  http://grouplens.org/datasets/movielens/
content:  http://grouplens.org/datasets/movielens/
attributes:  {'href': 'http://grouplens.org/datasets/movielens/'}
----
links:  <a class="anchor-link" href="#Frecuencia-de-votos">¶</a>
url:  #Frecuencia-de-votos
content:  ¶
attributes:  {'class': ['anchor-link'], 'href': '#Frecuencia-de-votos'}
----
links:  <a class="anchor-link" href="#References">¶</a>
url:  #References
content:  ¶
attributes:  {'class': ['anchor-link'], 'href': '#References'}
----
links:  <a href="http://grouplens.org/datasets/movielens/">MovieLens Dataset</a>
url:  http://grouplens.org/datasets/movielens/
content:  MovieLens Dataset
attributes:  {'href': 'http://grouplens.org/datasets/movielens/'}
----
lin

El ejemplo anterior imprime todo lo que se encuentra en la etiqueta `<a>`, pero en algunos casos no se corresponde con un enlace a un sitio web, sino que es una referencia a un punto del mismo documento. Esos enlaces tienen un atributo adicional llamado `class`.

El siguiente fragmento de código recupera  solo los enlaces a páginas internas, que como puedes ver en la ejecución anterior, se corresponden con fragmentos del HTML con la etiqueta __a__,  seguido de la clase __anchor-link__.   




In [69]:
tags = soup('a')
enlaces = soup.find_all("a", {'class':'anchor-link'})

print ('----')
for links in enlaces:
    print ('links: ', links) 
    print ('url: ', links.get('href'))
    print ('content: ', links.contents[0])
    print ('attributes: ', links.attrs)
    print ('----')

----
links:  <a class="anchor-link" href="#Movilens-1M">¶</a>
url:  #Movilens-1M
content:  ¶
attributes:  {'href': '#Movilens-1M', 'class': ['anchor-link']}
----
links:  <a class="anchor-link" href="#Frecuencia-de-votos">¶</a>
url:  #Frecuencia-de-votos
content:  ¶
attributes:  {'href': '#Frecuencia-de-votos', 'class': ['anchor-link']}
----
links:  <a class="anchor-link" href="#References">¶</a>
url:  #References
content:  ¶
attributes:  {'href': '#References', 'class': ['anchor-link']}
----


##### Ejercicio: 
Escribe el fragmento de código capaz de recuperar solo los enlaces a páginas externas. Como sugerencia, usa la función __has_attr__ con el atributo __class__  sobre cada enlace de la lista de enlaces.

In [18]:
# Solución

enlaces = soup.find_all("a")

print ('----')
for links in enlaces:
    if links.has_attr('class'):
        pass
    else:
        print ('links: ', links) 
        print ('url: ', links.get('href'))
        print ('content: ', links.contents[0])
        print ('attributes: ', links.attrs)
        print ('----')


----
links:  <a href="http://grouplens.org/datasets/movielens/">http://grouplens.org/datasets/movielens/</a>
url:  http://grouplens.org/datasets/movielens/
content:  http://grouplens.org/datasets/movielens/
attributes:  {'href': 'http://grouplens.org/datasets/movielens/'}
----
links:  <a href="http://grouplens.org/datasets/movielens/">MovieLens Dataset</a>
url:  http://grouplens.org/datasets/movielens/
content:  MovieLens Dataset
attributes:  {'href': 'http://grouplens.org/datasets/movielens/'}
----
links:  <a href="http://matplotlib.org/">LibrerÃ­a grÃ¡fica matplotlib</a>
url:  http://matplotlib.org/
content:  LibrerÃ­a grÃ¡fica matplotlib
attributes:  {'href': 'http://matplotlib.org/'}
----
links:  <a href="http://pandas.pydata.org/pandas-docs/stable/visualization.html">Plotting pandas</a>
url:  http://pandas.pydata.org/pandas-docs/stable/visualization.html
content:  Plotting pandas
attributes:  {'href': 'http://pandas.pydata.org/pandas-docs/stable/visualization.html'}
----
links:  <

#### 2. Recuperar los datos de la tabla

Las tablas de HTML se definen con tres etiquetas: `<table>` para crear la tabla, `<tr>` para crear cada fila y `<td>` para crear cada columna.


In [20]:
from IPython.display import HTML
s = """
<table>
    <tr>
      <th><strong>Curso</strong></th>
      <th><strong>Horas</strong></th>
    </tr>
 
    <tr>
      <td>CSS</td>
      <td>20</td>
    </tr>
 
    <tr>
      <td>HTML</td>
      <td>20</td>
    </tr>
</table>"""
HTML(s)

Curso,Horas
CSS,20
HTML,20


__Ejemplo:__ Queremos recuperar los datos de la tabla que aparece en la página  [Estadística con datos de movilens](http://gpd.sip.ucm.es/yolanda/formacionContinua/EV2016_pelis.html)

En este caso utilizamos el método __find_all()__ que lo que hace es coger todos los fragmentos del HTML que correpondan a las etiquetas __tr__ y __td__.
 

In [12]:
content = {}
# Obtenemos cada una de las filas de la tabla 
rows = soup.find_all('tr')
for r in rows:
    #Seleccionamos las celdas de cada una de las filas de la tabla (td)
    celdas=r.find_all('td')
    
    # ignoramos la primera celda, que no tiene elementos td sino th 
    # (ver HTML de la página web)
    if len(celdas)>0:           
        titulo = celdas[0].string
        genero = celdas[1].string
        valoraciones_5 =  int(celdas[5].string)
        content[titulo] = {'genero' : genero, 'Cinco Estrellas': valoraciones_5}

content

{'Ace Ventura: When Nature Calls (1995)': {'Cinco Estrellas': 51,
  'genero': 'Comedy'},
 'American President, The (1995)': {'Cinco Estrellas': 492,
  'genero': 'Comedy|Drama|Romance'},
 'Assassins (1995)': {'Cinco Estrellas': 29, 'genero': 'Thriller'},
 'Balto (1995)': {'Cinco Estrellas': 36, 'genero': "Animation|Children's"},
 'Casino (1995)': {'Cinco Estrellas': 277, 'genero': 'Drama|Thriller'},
 'City of Lost Children, The (1995)': {'Cinco Estrellas': 141,
  'genero': 'Adventure|Sci-Fi'},
 'Copycat (1995)': {'Cinco Estrellas': 135, 'genero': 'Crime|Drama|Thriller'},
 'Cutthroat Island (1995)': {'Cinco Estrellas': 24,
  'genero': 'Action|Adventure|Romance'},
 'Dracula: Dead and Loving It (1995)': {'Cinco Estrellas': 22,
  'genero': 'Comedy|Horror'},
 'Father of the Bride Part II (1995)': {'Cinco Estrellas': 80,
  'genero': 'Comedy'},
 'Four Rooms (1995)': {'Cinco Estrellas': 51, 'genero': 'Thriller'},
 'Get Shorty (1995)': {'Cinco Estrellas': 553,
  'genero': 'Action|Comedy|Drama'},

##### Ejercicio:
Para cada película queremos recuperar también el número total de valoraciones obtenidas. 
Modificar el código anterior para generar un diccionario donde la clave sea la película, y el valor sea otro diccionario con tres claves: 'Seis Estrellas', 'Género' y 'Total valoraciones'.


In [14]:
# Solución:

content = {}
# Obtenemos cada una de las filas de la tabla 
rows = soup.find_all('tr')
for r in rows:
    #Seleccionamos las celdas de cada una de las filas de la tabla (td)
    celdas=r.find_all('td')
    
    # ignoramos la primera celda, que no tiene elementos td sino th 
    # (ver HTML de la página web)
    if len(celdas)>0:           
        titulo = celdas[0].string
        genero = celdas[1].string
        valoraciones_5 =  int(celdas[5].string)
        restoVal = int(celdas[5].string) + int(celdas[4].string) + int(celdas[3].string)
        content[titulo] = {'genero' : genero, 'Cinco Estrellas': valoraciones_5, 
                           'Total_val' :restoVal  }

content

{'Ace Ventura: When Nature Calls (1995)': {'Cinco Estrellas': 51,
  'Total_val': 267,
  'genero': 'Comedy'},
 'American President, The (1995)': {'Cinco Estrellas': 492,
  'Total_val': 816,
  'genero': 'Comedy|Drama|Romance'},
 'Assassins (1995)': {'Cinco Estrellas': 29,
  'Total_val': 108,
  'genero': 'Thriller'},
 'Balto (1995)': {'Cinco Estrellas': 36,
  'Total_val': 82,
  'genero': "Animation|Children's"},
 'Casino (1995)': {'Cinco Estrellas': 277,
  'Total_val': 515,
  'genero': 'Drama|Thriller'},
 'City of Lost Children, The (1995)': {'Cinco Estrellas': 141,
  'Total_val': 231,
  'genero': 'Adventure|Sci-Fi'},
 'Copycat (1995)': {'Cinco Estrellas': 135,
  'Total_val': 322,
  'genero': 'Crime|Drama|Thriller'},
 'Cutthroat Island (1995)': {'Cinco Estrellas': 24,
  'Total_val': 94,
  'genero': 'Action|Adventure|Romance'},
 'Dracula: Dead and Loving It (1995)': {'Cinco Estrellas': 22,
  'Total_val': 101,
  'genero': 'Comedy|Horror'},
 'Father of the Bride Part II (1995)': {'Cinco Estr

-----------------

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />